In [1]:
#Basic imports
import pandas as pd
import numpy as np
import pickle
from collections import defaultdict
from os import environ, getcwd
import sys

#Classifier imports
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier

#ML framework imports
from sklearn.metrics import auc, roc_auc_score, precision_recall_curve, precision_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

#Import utils functions
curr_dir = !pwd
sys.path.append(curr_dir[0]+"/utils")
from prop_threshold_funcs import create_negatives_datasets, create_positives_datasets
from prediction_general_funcs import ligands, score_cols_suffix, get_features_cols
from CV_funcs import add_domain_name_from_table_idx, calc_CV_idx_iterative

from IPython.core.display import HTML
HTML("<style>.container { width:100% !important; }</style>")

### Reading the input dataset

In [19]:
curr_dir = !pwd
pfam_version = "31"
datafile_date = "06.20.18"
input_path = curr_dir[0]+"/domains_similarity/filtered_features_table/"
filename = "windowed_positions_features_mediode_filter_"+datafile_date+".csv"
out_dir = "mediode_NegLigand_NoFilter"

#flags for creating negatives
zero_prop = True
no_prop = True
all_ligands = False
prec_th = 0.25
folds_num = 10

#Features table
features_all = pd.read_csv(input_path+filename, sep='\t', index_col=0)
#Features columns names, without the labels (the binding scores)
features_cols = get_features_cols(features_all)

print "all samples positions #: "+str(features_all.shape[0])

#CV splits dictionary
with open(curr_dir[0]+"/CV_splits/pfam-v"+pfam_version+"/domain_"+str(folds_num)+"_folds_"+str(prec_th)+"_prec_dict.pik", 'rb') as handle:
        splits_dict = pickle.load(handle)

all samples positions #: 42535


#### Dataset of negative examples

In [5]:
ligands_negatives_df = create_negatives_datasets(zero_prop, no_prop, features_all, features_cols, all_ligands)

dna non-binding #:41680
dnabase non-binding #:42089
dnabackbone non-binding #:41689
rna non-binding #:41613
rnabase non-binding #:41828
rnabackbone non-binding #:41619
peptide non-binding #:38794
ion non-binding #:37525
metabolite non-binding #:37463
sm non-binding #:30978


#### Datasets of positive examples by ligand

In [6]:
ligands_positives_df = create_positives_datasets(prec_th, features_all, features_cols, all_ligands)

dna #: 543
dnabase #: 284
dnabackbone #: 543
rna #: 360
rnabase #: 246
rnabackbone #: 346
peptide #: 1467
ion #: 1500
metabolite #: 1427
sm #: 2340


/home/anat/anaconda2/lib/python2.7/site-packages/pandas/core/frame.py:1997: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


### Reading env input for downsampler technique, ligand and classifier

In [14]:
#Reading the ligand input
try:
    ligand = environ['ligand']
except:
    ligand = "dna"
print "ligand = "+ligand
    
#Reading the downsampler input
try: 
    fold = environ['fold']
except:
    fold = "1"
print "fold = "+fold

#Reading the classifier input
try: 
    classifier_method = environ['classifier']
except:
    classifier_method = "Logistic"
print "classifier_method = "+classifier_method

ligand = dna
fold = 1
classifier_method = Logistic


### Models tested (and their hyper-parameters)

In [15]:
classifiers = {}
if (classifier_method == "XGB"):
    ligand_pos = ligands_positives_df[ligand].shape[0]
    ligand_neg = ligands_positives_df[ligand].shape[0]
    scale_weight = ligand_neg/float(ligand_pos)
    classifiers["XGB"] = XGBClassifier(n_estimators=1000, n_jobs=-1, random_state=0, max_depth=6, min_child_weight=0.05, colsample_bytree=0.5, scale_pos_weight=scale_weight)
elif (classifier_method == "RF"):
    classifiers["RF"] = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=0)  
elif(classifier_method == "Logistic"):
    classifiers["Logistic"] = LogisticRegression(C=0.1, random_state=0, n_jobs=-1)
elif (classifier_method == "KNN"):
    classifiers["KNN"] = KNeighborsClassifier(n_neighbors=100, n_jobs=-1)
elif (classifier_method == "ADA"):
    classifiers["ADA"] = AdaBoostClassifier(n_estimators=1000, random_state=0)
elif (classifier_method == "SVM"):
    classifiers["SVM"] = SVC(kernel='rbf', probability=True, random_state=0)

In [16]:
def compute_per_domain_auc(y_test, pred_probs, domain_pred_dict, pred_idx, classifier):
    """
    Compute the average per_domain auc and auprc for the test set
    """
    
    y_test_copy = y_test.copy(deep=True)
    y_test_copy["pred_probs"] = pred_probs
    
    domain_auc_list = []
    domain_auprc_list = []
    domain_auprc_ratio_list = []
    domain_name_list = []
    
    idx = y_test.index
    y_test_copy["domain_name"] = [x[:x.rfind("_")] for x in idx]
    domains_list = y_test_copy["domain_name"].unique().tolist()
    
        
    for domain_name in domains_list:
        
        #Get only the domain positions
        domain_df = y_test_copy[y_test_copy["domain_name"] == domain_name]

        #Find the binding positions of this domain
        bind_list = domain_df[domain_df["label"] == 1].index
        bind_idx = [int(x[len(domain_name)+1:]) for x in bind_list]
        bind_num = len(bind_idx)
        if (bind_num == 0):
            #No binding positions in this domain - skipping"
            continue
        
        domain_pred_dict["obs"].extend(domain_df["label"])
        domain_pred_dict["prob"].extend(domain_df["pred_probs"])
        fold_list = [pred_idx] * len(domain_df["pred_probs"])
        domain_pred_dict["fold"].extend(fold_list)
        model_list = [classifier] * len(domain_df["pred_probs"])
        domain_pred_dict["model"].extend(model_list)
        domain_str_list = [domain_name] * len(domain_df["pred_probs"])
        domain_pred_dict["domain"].extend(domain_str_list)
    
        #Compute domain AUC
        domain_auc = roc_auc_score(domain_df["label"], domain_df["pred_probs"])
        domain_auc_list.append(domain_auc)
        #Compute domain AUPRC
        precision, recall, thresholds = precision_recall_curve(domain_df["label"], domain_df["pred_probs"])
        domain_auprc = auc(recall, precision)
        domain_auprc_list.append(domain_auprc)
        #Add positives fraction to list
        pos_frac_ratio = bind_num/float(domain_df.shape[0])
        #Add ratio of AUPRC and positives fraction to list
        domain_auprc_ratio_list.append(domain_auprc/float(pos_frac_ratio))
        #Add domain name for AUC/AUPRC/Ratio tables
        domain_name_list.append(domain_name)
        
    #Compute the means for the lists 
    domain_auc_mean = np.mean(domain_auc_list)
    domain_auprc_mean = np.mean(domain_auprc_list)
    domain_auprc_ratio_mean = np.mean(domain_auprc_ratio_list)
    
    return (domain_auc_mean, domain_auprc_mean, domain_auprc_ratio_mean, domain_auc_list, domain_auprc_list, domain_auprc_ratio_list, domain_name_list)

In [11]:
def test_model_iterative_fixed(pred_dict, domain_pred_dict, auc_dict, auprc_dict, domain_auc_mean_dict, domain_auprc_mean_dict, domain_auprc_ratio_mean_dict, domain_auc_dict, domain_auprc_dict, domain_auprc_ratio_dict,
                               ligand_bind_features, ligand_negatives_features, ligand_name, features=[]):
    
    """
    Test different models in 10-folds cross-validation.
    """
    
    #Default: Exclude no features
    if len(features) == 0:
        features = np.ones([ligand_bind_features.shape[1],]).astype(bool)
        
    #Arranging the features table by the CV order, for each model
    features_pred_dfs = dict.fromkeys(classifiers.keys())
    
    models_req_scaling = ["SVM", "KNN", "Logistic"]

    classifier = classifier_method
    model = classifiers[classifier]
    features_pred_dfs[classifier] = pd.DataFrame()

    #Create X and y with included features
    X = pd.concat([ligand_bind_features.iloc[:,features], ligand_negatives_features.iloc[:,features]])
    y = [1] * ligand_bind_features.shape[0]
    y.extend([0] * ligand_negatives_features.shape[0])
    y = np.array(y)
    y_df = pd.DataFrame(y)
    y_df.index = X.index
    y_df.columns = ["label"]
    
    #Get the fold indices
    cv_idx = calc_CV_idx_iterative(X, splits_dict)
    k = (int(fold)-1)
    
    pred_idx = k+1
    print "fold #: "+str(pred_idx)
    test_index = cv_idx[k]["test"]
    train_index = cv_idx[k]["train"]
    X_train, X_test = X.loc[train_index,:], X.loc[test_index,:]
    y_train, y_test = y_df.loc[train_index,:], y_df.loc[test_index,:]
    
    if (classifier in models_req_scaling):
        cols = X_train.columns
        scaler = StandardScaler() 
        #scale only using the training data
        scaler.fit(X_train) 
        X_train = pd.DataFrame(scaler.transform(X_train))
        # apply same transformation to test data
        X_test = pd.DataFrame(scaler.transform(X_test))
        #Restoring indices after scaling
        X_train.index = train_index 
        X_test.index = test_index 
        #Restoring features names
        X_train.columns = cols
        X_test.columns = cols

    #No down-sampling
    X_train_sampled = X_train
    y_train_sampled = y_train

    #fit to training data
    model = classifiers[classifier]
    model.fit(X_train_sampled, y_train_sampled["label"])

    probs_list = []
    probs = model.predict_proba(X_test)
    for l in probs:
        probs_list.append(l[1])

    pred_dict["obs"].extend(y_test["label"])
    pred_dict["prob"].extend(probs_list)
    fold_list = [pred_idx] * len(probs_list)
    pred_dict["fold"].extend(fold_list)

    model_list = [classifier] * len(probs_list)
    pred_dict["model"].extend(model_list)

    #Update auc auprc dictionaries
    auc_dict[classifier].append(roc_auc_score(y_test["label"], probs[:, 1]))
    precision, recall, _ = precision_recall_curve(y_test["label"], probs[:, 1])
    auprc_dict[classifier].append(auc(recall, precision))
    
    #Compute per domain AUC and AUPRC
    (domain_auc_mean, domain_auprc_mean, domain_auprc_ratio_mean, domain_auc_list, domain_auprc_list, domain_auprc_ratio_list, domain_name_list) = compute_per_domain_auc(y_test, probs[:, 1], domain_pred_dict,pred_idx, classifier)
    
    #Update relevant dictionaries for per-domain folds mean
    domain_auc_mean_dict[classifier].append(domain_auc_mean)
    domain_auprc_mean_dict[classifier].append(domain_auprc_mean)
    domain_auprc_ratio_mean_dict[classifier].append(domain_auprc_ratio_mean)
    
    #Update relevant dictionaries for per-domain individual metrices scores
    domain_auc_dict[classifier].extend(domain_auc_list)
    domain_auc_dict["domain"].extend(domain_name_list)
    domain_auprc_dict[classifier].extend(domain_auprc_list)
    domain_auprc_dict["domain"].extend(domain_name_list)
    domain_auprc_ratio_dict[classifier].extend(domain_auprc_ratio_list)
    domain_auprc_ratio_dict["domain"].extend(domain_name_list)
    
    #Update features table
    features_pred_dfs[classifier] = features_pred_dfs[classifier].append(X_test)
    pred_idx += 1

    print "AUC = "+str(auc_dict[classifier][-1])
    print "AUPRC = "+str(auprc_dict[classifier][-1])
    print "domain AUC mean = "+str(domain_auc_mean_dict[classifier][-1])
    print "domain AUPRC mean = "+str(domain_auprc_mean_dict[classifier][-1])
    print "domain AUPRC ratio mean = "+str(domain_auprc_ratio_mean_dict[classifier][-1])

    print "Finished "+ligand+" "+classifier+" fold: "+fold
    
    return features_pred_dfs

### Test model functions

In [17]:
def combine_features_predictions(ligand, ordered_features, pred_df):
    
    pred_res = pred_df.copy(deep=True)
    for classifier in classifiers.keys():
        classifier = classifier_method
        model_pred = pred_res[pred_res["model"] == classifier]
        model_pred.index = ordered_features[classifier].index
        
        #Creating the combined table
        features_pred = pd.concat([ordered_features[classifier], model_pred], axis=1)
        
        #Saving
        features_pred.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/features_pred_tables/"+ligand+"_"+classifier+"_features_pred.csv", sep='\t')
        break

#### Predict for each ligand seperatelly

In [24]:
%%time

#Initialize dictionary
pred_dict = defaultdict(list)
domain_pred_dict = defaultdict(list)
auc_dict = defaultdict(list)
auprc_dict = defaultdict(list)
domain_auc_mean_dict = defaultdict(list)
domain_auprc_mean_dict = defaultdict(list)
domain_auprc_ratio_mean_dict = defaultdict(list)
domain_auc_dict = defaultdict(list)
domain_auprc_dict = defaultdict(list)
domain_auprc_ratio_dict = defaultdict(list)
downsample_method = "NoDown"

ordered_features = test_model_iterative_fixed(pred_dict, domain_pred_dict, auc_dict, auprc_dict, domain_auc_mean_dict, domain_auprc_mean_dict, domain_auprc_ratio_mean_dict, domain_auc_dict, domain_auprc_dict, domain_auprc_ratio_dict, ligands_positives_df[ligand], ligands_negatives_df[ligand], ligand)


pred_df = pd.DataFrame.from_dict(pred_dict)
domain_pred_df = pd.DataFrame.from_dict(domain_pred_dict)
#global matrics dfs
auc_df = pd.DataFrame.from_dict(auc_dict)
auprc_df = pd.DataFrame.from_dict(auprc_dict)
#per domain mean dfs
domain_auc_mean_df = pd.DataFrame.from_dict(domain_auc_mean_dict)
domain_auprc_mean_df = pd.DataFrame.from_dict(domain_auprc_mean_dict)
domain_auprc_ratio_mean_df = pd.DataFrame.from_dict(domain_auprc_ratio_mean_dict)
#per domain dfs
domain_auc_df = pd.DataFrame.from_dict(domain_auc_dict)
domain_auprc_df = pd.DataFrame.from_dict(domain_auprc_dict)
domain_auprc_ratio_df= pd.DataFrame.from_dict(domain_auprc_ratio_dict)

#Save to file
pred_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w.csv", sep='\t')
domain_pred_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_d.csv", sep='\t')

auc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_auc.csv", sep='\t')
auprc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_auprc.csv", sep='\t')

domain_auc_mean_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_dm_auc.csv", sep='\t')
domain_auprc_mean_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_dm_auprc.csv", sep='\t')
domain_auprc_ratio_mean_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_10w_dm_auprc_ratio.csv", sep='\t')

domain_auc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_d_auc.csv", sep='\t')
domain_auprc_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_d_auprc.csv", sep='\t')
domain_auprc_ratio_df.to_csv(curr_dir[0]+"/pred_AUC_AUPRC/"+out_dir+"/"+downsample_method+"/"+datafile_date+"_domain_CV/per_fold/"+ligand+"_"+classifier_method+"_fold"+fold+"_"+str(folds_num)+"w_d_auprc_ratio.csv", sep='\t')

#Combine features and pred results to a unified table
#combine_features_predictions(ligand, ordered_features, pred_df)

print "Finished ligand "+ligand

fold #: 1
AUC = 0.879474373229
AUPRC = 0.0882699579639
domain AUC mean = 0.778996295897
domain AUPRC mean = 0.416551793322
domain AUPRC ratio mean = 2.69965598531
Finished dna Logistic fold: 1
Finished ligand dna
CPU times: user 48.1 s, sys: 952 ms, total: 49 s
Wall time: 48 s
